In [1]:
import xarray as xr
import s3fs
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
from datetime import datetime
import numpy as np
from dask.distributed import Client, LocalCluster

In [2]:
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0", client_kwargs=dict(endpoint_url="https://rgw.met.no"))

In [5]:
s3.ls('escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration')[::-1]

['escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/zip',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20111004-v5.4.nc',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20111003-v5.4.nc',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20111002-v5.4.nc',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20111001-v5.4.nc',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20110930-v5.4.nc',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20110929-v5.4.nc',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20110928-v5.4.nc',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20110927-v5.4.nc',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20110926-v5.4.nc',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20110925-v5.4.nc',
 'escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20110924-v5.4.nc',
 'e

In [19]:
def paths_to_datetimeindex(paths):
    return  [datetime.strptime(date.split('/')[-1].split('-')[3], '%Y%m%d') for date in paths]

In [4]:
def read_daily_files(year):
    for month in range (1,13):
        remote_files = 's3://escience2022/Antoine/AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-%i%s*.nc'%(year,str(month).zfill(2))
        remote_files = s3.glob(remote_files)
        fileset = [s3.open(file) for file in remote_files]
        time_var = xr.Variable('time', paths_to_datetimeindex(remote_files))
        dset =xr.open_mfdataset(fileset, concat_dim="time", combine = "nested")
        dset = dset.assign_coords(time=time_var)
        dset.to_netcdf('AMSR2_ASI_sea_ice_concentration_%i%s.nc'%(year,str(month).zfill(2)))
        del remote_files, dset, time_var

In [5]:
[read_daily_files(year) for year in range(2016,2020)]

[None, None, None, None]

In [3]:
def paths_to_datetimeindex(paths):
    return  [datetime.strptime(date.split('/')[-1].split('-')[2], '%Y%m%d') for date in paths]

In [4]:
def read_daily_files(year):
    for month in range (1,13):
        remote_files = 's3://escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-%i%s*.nc'%(year,str(month).zfill(2))
        remote_files = s3.glob(remote_files)
        fileset = [s3.open(file) for file in remote_files]
        time_var = xr.Variable('time', paths_to_datetimeindex(remote_files))
        dset =xr.open_mfdataset(fileset, concat_dim="time", combine = "nested")
        dset = dset.assign_coords(time=time_var)
        dset.to_netcdf('AMSR-E_ASI_sea_ice_concentration_%i%s.nc'%(year,str(month).zfill(2)))
        del remote_files, dset, time_var

In [ ]:
[read_daily_files(year) for year in range(2011,2012)]

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
